In [1]:
# 1. import libraries
import requests # to download html code
from bs4 import BeautifulSoup # to navigate through the html code
import pandas as pd
import numpy as np
import re

In [2]:
# We use the dataset from kaggle https://www.kaggle.com/datasets/saurabhshahane/music-dataset-1950-to-2019?resource=download
# containing over 28 k songs between 1950 and 2019

songs = pd.read_csv("C:/Users/Sabrina/Desktop/TestDA/Week_6/Day_1/Afternoon/lab-not-hot-songs/tcc_ceds_music.csv")
songs.head()

,Unnamed: 0,artist_name,track_name,release_date,genre,lyrics,len,dating,violence,world/life,...,sadness,feelings,danceability,loudness,acousticness,instrumentalness,valence,energy,topic,age
0,0,mukesh,mohabbat bhi jhoothi,1950,pop,hold time feel break feel untrue convince spea...,95,0.000598,0.063746,0.000598,...,0.380299,0.117175,0.357739,0.454119,0.997992,0.901822,0.339448,0.137110,sadness,1.0
1,4,frankie laine,i believe,1950,pop,believe drop rain fall grow believe darkest ni...,51,0.035537,0.096777,0.443435,...,0.001284,0.001284,0.331745,0.647540,0.954819,0.000002,0.325021,0.263240,world/life,1.0
2,6,johnnie ray,cry,1950,pop,sweetheart send letter goodbye secret feel bet...,24,0.002770,0.002770,0.002770,...,0.002770,0.225422,0.456298,0.585288,0.840361,0.000000,0.351814,0.139112,music,1.0
3,10,pérez prado,patricia,1950,pop,kiss lips want stroll charm mambo chacha merin...,54,0.048249,0.001548,0.001548,...,0.225889,0.001548,0.686992,0.744404,0.083935,0.199393,0.775350,0.743736,romantic,1.0
4,12,giorgos papadopoulos,apopse eida oneiro,1950,pop,till darling till matter know till dream live ...,48,0.001350,0.001350,0.417772,...,0.068800,0.001350,0.291671,0.646489,0.975904,0.000246,0.597073,0.394375,romantic,1.0


In [3]:
songs.shape

(28372, 31)

In [4]:
# We isolate only the columns "artist_name"and "track_name" and perform some changes in the columns to align them with the df hot_songs

not_hot = songs[["artist_name","track_name"]]
not_hot= not_hot.rename(columns={"artist_name": "artist", "track_name": "title"})
display(not_hot.head())
display(not_hot.shape)

,artist,title
0,mukesh,mohabbat bhi jhoothi
1,frankie laine,i believe
2,johnnie ray,cry
3,pérez prado,patricia
4,giorgos papadopoulos,apopse eida oneiro


(28372, 2)

In [5]:
# We pick a sample of only 3000 random songs to make the file a bit "lighter" faster to process
not_hot = not_hot.sample(n=3000, random_state=1)
not_hot.shape

(3000, 2)

In [6]:
# We load the hot_songs df we created in the previous lab:

hot_songs= pd.read_csv("./hot_songs_09_jan-Copy1.csv")
hot_songs.head()

,artist,title
0,Jack Harlow,Lovin On Me
1,Taylor Swift,Cruel Summer
2,Tate McRae,Greedy
3,Doja Cat,Paint The Town Red
4,Zach Bryan Featuring Kacey Musgraves,I Remember Everything


In [13]:
hot_songs.shape

(100, 2)

In [7]:
# We look for potential duplicates in both the hot_song df:

hot_songs.groupby("title").size().sort_values(ascending=False)

title
23                                   1
Pretty Little Poison                 1
Slime You Out                        1
Segun Quien                          1
Save Me The Trouble                  1
                                    ..
Hey Driver                           1
Harley Quinn                         1
Greedy                               1
Great Gatsby                         1
You're Losing Me (From The Vault)    1
Length: 100, dtype: int64

In [8]:
# We concatenate both not_hot and hot_songs df to check for duplicates before applying a function to remove them all
checking_df = pd.concat([not_hot, hot_songs], ignore_index=True)
checking_df.head()

,artist,title
0,glenn miller,the little man who wasn't there
1,misfits,american psycho
2,elliott smith,somebody that i used to know
3,june carter cash,juke box blues
4,"emerson, lake & palmer","karn evil 9 1st impression, pt. 1"


In [9]:
# We check the total shape of the concatenated df:
checking_df.shape

(3100, 2)

In [10]:
checking_df.isna().sum()

artist    0
title     0
dtype: int64

In [11]:
checking_df.duplicated().sum()

0

In [26]:
def remove_duplicated_concat_hotsongs(df_hot: pd.DataFrame, df_not_hot: pd.DataFrame) -> pd.DataFrame:
    """
    Creates copies of the input DataFrames (df_hot and df_not_hot) to avoid modifying the originals.
    Removes rows present in df_hot from df_not_hot, ensuring only unique rows are retained.

    Parameters:
    - df_hot (pd.DataFrame): The DataFrame with certain rows to be removed.
    - df_not_hot (pd.DataFrame): The DataFrame from which rows present in df_hot will be removed.

    Returns:
    - pd.DataFrame: A new DataFrame containing the unique rows from df_not_hot,
      excluding rows present in df_hot.
    """
   # Create copies of the input DataFrames to avoid modifying the originals
    df2 = df_hot.copy()
    df3 = df_not_hot.copy()

    # Remove rows present in df_hot from df_not_hot (df3)
    df4 = df3[~df3.isin(df2)].dropna(ignore_index=True)
    
    
    return df4


In [36]:
def remove_duplicates(df1, df2):
  
    # Remove duplicated rows from df1
    result_df = df1[~df1.isin(df2)].dropna().reset_index(drop=True)

    return result_df

In [37]:
final_df_2 = remove_duplicates(not_hot, hot_songs)
display(final_df_2)

,artist,title
0,glenn miller,the little man who wasn't there
1,misfits,american psycho
2,elliott smith,somebody that i used to know
3,june carter cash,juke box blues
4,"emerson, lake & palmer","karn evil 9 1st impression, pt. 1"
...,...,...
2995,guns n' roses,reckless life
2996,t. rex,pain and love
2997,greta van fleet,brave new world
2998,g. love & special sauce,i-76


In [27]:
final_df = remove_duplicated_concat_hotsongs(hot_songs, not_hot)
display(final_df)

,artist,title
0,glenn miller,the little man who wasn't there
1,misfits,american psycho
2,elliott smith,somebody that i used to know
3,june carter cash,juke box blues
4,"emerson, lake & palmer","karn evil 9 1st impression, pt. 1"
...,...,...
2995,guns n' roses,reckless life
2996,t. rex,pain and love
2997,greta van fleet,brave new world
2998,g. love & special sauce,i-76


In [28]:
final_df.to_csv("./final_df.csv", index=False)